In [10]:
import torch 
import numpy as np
from datasets import load_dataset
from IPython.display import display, Audio

In [9]:
# from silero_vad import load_silero_vad, read_audio, get_speech_timestamps
# model = load_silero_vad()

import torch
model, utils = torch.hub.load(repo_or_dir='snakers4/silero-vad', model='silero_vad', force_reload=True)
get_speech_timestamps  = utils[0]

Downloading: "https://github.com/snakers4/silero-vad/zipball/master" to /home/jovyan/.cache/torch/hub/master.zip


In [2]:
from pyannote.audio import Model
model = Model.from_pretrained("pyannote/segmentation-3.0")
from pyannote.audio.pipelines import VoiceActivityDetection
pipeline = VoiceActivityDetection(segmentation=model)
HYPER_PARAMETERS = {
  # remove speech regions shorter than that many seconds.
  "min_duration_on": 0.0,
  # fill non-speech regions shorter than that many seconds.
  "min_duration_off": 0.0
}
pipeline.instantiate(HYPER_PARAMETERS)

In [3]:
ds = load_dataset("hsekhalilian/persian-youtube")

Resolving data files:   0%|          | 0/54 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/54 [00:00<?, ?it/s]

Loading dataset shards:   0%|          | 0/54 [00:00<?, ?it/s]

In [4]:
dataset = ds["train"].select(range(10000, 10200))

In [5]:
for sample in dataset.select(range(10)):
    print("Sentence:", sample['sentence'])
    display(Audio(sample['audio']['array'], rate=sample['audio']['sampling_rate']))
    break

Sentence: هم آزمایش سلاح‌های مختلف هست


In [16]:
chunk_size = 15
overlap = 3
sampling_rate = dataset[0]['audio']['sampling_rate']  # assuming consistent rate

step = chunk_size - overlap
if step <= 0:
    raise ValueError("Overlap must be smaller than chunk size")

for i in range(0, len(dataset), step):
    chunk = dataset.select(range(i, min(i + chunk_size, len(dataset))))
    audios = [sample['audio']['array'] for sample in chunk]
    combined_audio = np.concatenate(audios)
    
    print(f"Samples {i} to {i + len(chunk) - 1}:")
    display(Audio(combined_audio, rate=sampling_rate))
    
    print("Sentences with time stamps:")
    current_time = 0.0
    for sample in chunk:
        audio_array = sample['audio']['array']
        duration = len(audio_array) / sampling_rate
        start_time = current_time
        end_time = current_time + duration
        print(f"- [{start_time:.2f}s - {end_time:.2f}s]: {sample['sentence']}")
        current_time = end_time
    # if i > 0:
    break

Samples 0 to 14:


Sentences with time stamps:
- [0.00s - 2.46s]: هم آزمایش سلاح‌های مختلف هست
- [2.46s - 4.58s]: از نارنجک و تفنگ و اینها
- [4.58s - 6.18s]: تا سلاح‌های میکروبی
- [6.18s - 7.87s]: طاعون، آبله مرغون
- [7.87s - 10.31s]: این ویروس‌ها رو پخش کنیم ببینیم چطوری می‌شه
- [10.31s - 11.68s]: بین آدما پخش کنیم
- [11.68s - 14.05s]: به بچه‌ها بدیم، به زنای حامله بدیم
- [14.05s - 16.22s]: ببینیم حالا رو بچه چه اثری می‌ذاره
- [16.22s - 18.82s]: زن حامله نیست، بارداری اجباری درست کنیم که
- [18.82s - 19.89s]: بررسی کنیم ببینیم
- [19.89s - 22.67s]: ویروس در دوره‌ی بارداری چیکار می‌کنه؟
- [22.67s - 24.46s]: سفلیس چطوری منتقل می‌شه؟
- [24.46s - 26.14s]: زنده‌شکافی کنیم ببینیم
- [26.14s - 28.46s]: داروهای مختلف چه اثری روی بدن داشته؟
- [28.46s - 29.87s]: حیرت انگیزه واقعاً


In [17]:
speech_timestamps = get_speech_timestamps(
    combined_audio, 
    model,
    sampling_rate=sampling_rate,
    return_seconds=True,
    

)
speech_timestamps

[{'start': 0.0, 'end': 29.870375}]

In [18]:
if isinstance(combined_audio, torch.Tensor):
    waveform = combined_audio.unsqueeze(0)  # shape becomes (1, 247716)
else:
    waveform = torch.tensor(combined_audio, dtype=torch.float32).unsqueeze(0)

vad_result = pipeline({"waveform": waveform, "sample_rate": 16000})
for segment in vad_result.itersegments():
    print(f"Speech from {segment.start:.2f}s to {segment.end:.2f}s")

Speech from 0.03s to 29.73s
